<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Nov  1 17:43:47 2017

@author: emg
"""

"""
 - Install the bigquery library with "pip install google-cloud-bigquery"

 - Go to the Google Cloud console: https://console.cloud.google.com
 - Create a project ('reddit-network' or whatever)
 - Create service account credentials with read privileges: https://console.cloud.google.com/apis/credentials
 - Download the credentials to the local dir and point the CREDENTIALS variable at them
 - Add the 'BigQuery Job User' role to the service account: https://console.cloud.google.com/iam-admin/iam
 
 - BigQuery Python API: https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html#tables
 - BigQuery API: https://cloud.google.com/bigquery/docs/tables
 - SQL docs: https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax
 - Parameterized query docs: https://cloud.google.com/bigquery/docs/parameterized-queries
 - Reddit table schema: https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.all

 - Install spacy
 - Download the english model: `python -m spacy download en`
"""

import scipy as sp
import spacy
import pandas as pd
import pickle
import gzip
from tqdm import tqdm
from google.cloud import bigquery
from logging import getLogger
from concurrent.futures import TimeoutError
import sys
import os
from pathlib import Path

PROJECT = 'reddit-network-184710'
CREDENTIALS = 'reddit-network-774059619c28.json'

CACHE = 'cache'

def client():
    return bigquery.Client.from_service_account_json(CREDENTIALS, project=PROJECT)


In [3]:
PROJECT = 'reddit-network-184710'
CREDENTIALS = 'reddit-network-774059619c28.json'

CACHE = 'cache'

def client():
    return bigquery.Client.from_service_account_json(CREDENTIALS, project=PROJECT)

def tables():
    c = client()
    ds = c.dataset('reddit_comments', 'fh-bigquery')
    return [t.table_id for t in c.list_dataset_tables(ds)]

def job(query, config=None, max_bytes=1e9):
    config = config or bigquery.QueryJobConfig()
    config.use_legacy_sql = False
    config.maximum_bytes_billed = int(max_bytes)
    
    print('Submitting query')
    j = client().query(query=query, job_config=config)
    with tqdm() as pbar:
        while True:
            try:
                j.result(timeout=1)
            except TimeoutError:                
                pbar.update(1)
            else:
                break
            
    return j

def unpack(j):
    print('Unpacking results')
    
    total = j.query_results().total_rows
    
    iterator = j.result()
    rows = []
    for row in tqdm(iterator, total=total):
        rows.append(row.values())
        
    columns = [c.name for c in iterator.schema]
    return pd.DataFrame(rows, None, columns)
    

def all_comments(table, subreddit, **kwargs):
    query = """select body, author, created_utc, parent_id, subreddit, score
               from `fh-bigquery.reddit_comments.{}`
               where (subreddit = @subreddit)
               """.format(table)
    
    config = bigquery.QueryJobConfig()
    config.query_parameters = (bigquery.ScalarQueryParameter('subreddit', 'STRING', subreddit))
    
    return unpack(job(query, config, **kwargs))

def sample_comments(table, size=10, **kwargs):
    query = """select subreddit, array_agg(struct(body, id) order by rand() desc limit @size) as agg
               from `fh-bigquery.reddit_comments.{}`
               group by subreddit""".format(table)
        
    config = bigquery.QueryJobConfig()
    config.query_parameters = (bigquery.ScalarQueryParameter('size', 'INT64', size),)
    
    result = unpack(job(query, config, **kwargs))
    
    # This could definitely be done faster :/
    samples = pd.concat(result.set_index('subreddit')['agg'].apply(pd.DataFrame).to_dict())
    
    return samples

def author_link_relation(table, **kwargs):
    query = """select subreddit, author, array_agg(link_id) as link_ids
               from `fh-bigquery.reddit_comments.{}`
               group by subreddit, author
               order by subreddit, author""".format(table)
    
    return unpack(job(query, **kwargs))

def lemmatize(samples, nlp):
    #TODO: This could be sped up with nlp.pipe
    strings = samples.body.add(' | ').groupby(level=0).sum()
    
    indices = []
    for i, s in enumerate(tqdm(strings)):
        indices.extend([(i, t.lemma) for t in nlp(s)])
    indices = sp.array(indices)
    
    rows = indices[:, 0]
    cols = indices[:, 1]
    vals = sp.ones_like(rows)
    indicators = sp.sparse.csc_matrix((vals, (rows, cols)), (len(samples), len(nlp.vocab.strings)+1))
    
    return {'subreddits': strings.index.tolist(), 
            'indicators': indicators}
    
def incidence_matrices(relation):
    results = {}
    with tqdm(total=len(relation)) as pbar:
        for i, (subreddit, group) in enumerate(relation.groupby('subreddit')):
            links = sp.array(sorted(sp.concatenate(group.link_ids.tolist())))
            authors = sp.array(sorted(group.author))
        
            rs, cs = [], []
            for _, row in group.iterrows():
                r = sp.searchsorted(authors, row.author)
                c = sp.searchsorted(links, row.link_ids)
                
                rs.append(sp.full_like(c, r))
                cs.append(c)
            rs, cs = sp.concatenate(rs), sp.concatenate(cs)
            vals = sp.ones_like(rs)
            
            incidence = sp.sparse.csr_matrix((vals, (rs, cs)), (len(authors), len(links))) 
            results[subreddit] = {'incidence': incidence, 'authors': authors, 'links': links}
            
            pbar.update(len(group))
        
    return results

def save(obj, name, subfolder):
    compressed = gzip.compress(pickle.dumps(obj), 1)
    
    print('Saving {:.1f}MB under "{}"'.format(len(compressed)/1e6, name))
    path = Path('.') / CACHE / subfolder / (name + '.gz')
    path.parent.mkdir(exist_ok=True, parents=True)
    path.write_bytes(compressed)
    
def load(name, subfolder):
    path = Path('.') / CACHE / subfolder / (name + '.gz')
    
    return pickle.loads(gzip.decompress(path.read_bytes()))
    
def save_incidence(incidence, name):
    save(incidence, name, 'incidence')

def load_incidence(name):
    return load(name, 'incidence')

def save_lemmatized(lemmatized, name):
    save(lemmatized, name, 'lemmatized')
    
def load_lemmatized(name):
    return load(name, 'lemmatized')

def example():
    nlp = spacy.load('en')
    
    names = tables()    
    name = '2007'
    
    samples = sample_comments(name)
    lemmatized = lemmatize(samples, nlp)
    save_lemmatized(lemmatized, name)
    
    relation = author_link_relation(name)
    incidence = incidence_matrices(relation)
    save_incidence(incidence, name)

In [1]:
import pandas as pd

df = pd.read_csv('cmv_sample.csv')
authors = list(df['author'].unique())
test_names = tuple(authors[:6])



authors = list(df['author'].unique())
test_names = tuple(authors[:6])

s = ''
for name in authors:
    s = s + "'" + name + "' ,"

s = s[:-2]

query = """SELECT subreddit, author, created_utc
            FROM `fh-bigquery.reddit_comments.2017_06`
            WHERE author IN ({})""".format(s)

query

"SELECT subreddit, author, created_utc\n            FROM `fh-bigquery.reddit_comments.2017_06`\n            WHERE author IN ('DeltaBot' ,'DantesCuttlefish' ,'Grunt08' ,'pickledoop' ,'2AFTW' ,'law-talkin-guy' ,'EnterprisingAss' ,'cdb03b' ,'Geralt_of_Rivia1' ,'Mitoza' ,'jerefromga' ,'[deleted]' ,'HadesSmiles' ,'meskarune' ,'polysyndetonic' ,'Manungal' ,'designate_event' ,'pillbinge' ,'Mr-Irrelevant-' ,'MPixels' ,'Kyras_Edelweiss' ,'Ardonpitt' ,'Chareon' ,'GhastlyKing' ,'MayaFey_' ,'thewoodendesk' ,'GalacticCow' ,'runcmc22' ,'Jasontheperson' ,'Dr_Scientist_' ,'SamBoosa58' ,'drpussycookermd' ,'camalew' ,'NoodlerOf88s' ,'Sentakusuru' ,'sunshinesasparilla' ,'alfiesolomons' ,'cleo_not_chloe' ,'PoloWearingMan' ,'AkumaBengoshi' ,'kebababab' ,'ACrusaderA' ,'Dark_Ice_Blade_Ninja' ,'TheManWhoWasNotShort' ,'Ahlekss' ,'cupcakesarethedevil' ,'rathyAro' ,'McKoijion' ,'Watawkichaw' ,'BrandonBradford' ,'ShiningConcepts' ,'Aubenabee' ,'3me_irl5me_irl' ,'PreacherJudge' ,'Mangiafuoc0' ,'jm0112358' ,'Plusis

In [4]:
config = bigquery.QueryJobConfig()
config.query_parameters = (bigquery.ScalarQueryParameter('size', 'INT64', 10),)

config = config or bigquery.QueryJobConfig()
config.use_legacy_sql = False
config.maximum_bytes_billed = int(5e9)

print('Submitting query')
j = client().query(query=query, job_config=config)
with tqdm() as pbar:
    while True:
        try:
            j.result(timeout=1)
        except TimeoutError:                
            pbar.update(1)
        else:
            break

Submitting query


35it [00:47,  1.30s/it]


In [6]:
print('Unpacking results')
    
total = j.query_results().total_rows

iterator = j.result()
rows = []
for row in tqdm(iterator, total=total):
    rows.append(row.values())

columns = [c.name for c in iterator.schema]
df = pd.DataFrame(rows, None, columns)

  0%|          | 0/7226803 [00:00<?, ?it/s]

Unpacking results


100%|██████████| 7226803/7226803 [06:49<00:00, 17668.01it/s]


In [7]:
df.shape

(7226803, 3)

In [8]:
sub = 'cmv'
df.to_csv('{}-17-06-cocomments.csv'.format(sub))
df.to_pickle('{}-17-06-cocomments.pkl'.format(sub))

cmv = pd.read_pickle('cmv-17-06-cocomments.pkl')
td = pd.read_pickle('td-17-06-cocomments.pkl')

def sub_counts(df):
    sub_counts = df['subreddit'].value_counts()
    sub_counts = pd.DataFrame(sub_counts).rename({'subreddit':'count'})
    sub_counts['portion'] = sub_counts['subreddit'] / sub_counts['subreddit'].sum()
    
    return sub_counts
    
cmv_counts = sub_counts(cmv)

cmv_counts.head(10)

cmv.head()

cmv_names = list(cmv['author'].unique())
td_names = list(td['author'].unique())
cmv_subs = list(cmv['subreddit'].unique())
td_subs = list(td['subreddit'].unique())

shared_authors = [name for name in cmv_names if name in td_names]
shared_subreddits = [subreddit for subreddit in cmv_subs if subreddit in td_subs]




print('TD has', len(td_names),'authors and', len(td_subs),'subreddits')
print()
print('CMV has', len(cmv_names),'authors and', len(cmv_subs),'subreddits')
print()
print('There are', len(shared_authors), 'shared authors and', len(shared_subreddits), 'shared subreddits')

x = cmv[cmv['author'].isin(shared_authors)]

y = td[td['author'].isin(shared_authors)]

print('CMV has', cmv.shape[0], 'total comments and', x.shape[0], 'by TD authors')
print()
print('TD has', td.shape[0], 'total comments and', y.shape[0], 'by TD authors')

cmv_comments = pd.read_csv('cmv_sample.csv')
cmv_comments.head()

td_comments = pd.read_csv('td_sample.csv')
td_comments.head()

sorted(cmv_comments['author'].unique()) == sorted(cmv_names)

sorted(td_comments['author'].unique()) == sorted(td_names)

union = pd.merge(cmv, td, how='inner')

remove = ['[deleted]','AutoModerator', 'WikiTextBot','teddyRbot']
botless = union[~union['author'].isin(remove)]

botless.shape

print('Removing bots, there are', len(botless['author'].unique()), 'shared authors and', len(botless['subreddit'].unique()), 'shared subreddits')

union['author'].unique()

botless['author'].value_counts()

botless['subreddit'].value_counts()